In [183]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import chain
from collections import Counter

import calendar

In [184]:
from pptx import Presentation 
from pptx.chart.data import ChartData
from pptx.dml.color import RGBColor
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
from pptx.enum.chart import XL_LABEL_POSITION
from pptx.util import Pt

from datetime import datetime
import os

import pandas as pd


from pptx.chart.data import ChartData
from pptx.enum.chart import XL_CHART_TYPE

In [185]:
def filehandle(path, sheetname):
    fh = pd.read_excel(path, sheet_name = sheetname)
#     Age
    if 'age' in fh.columns.str.lower():
        fh['Age'] = fh['AGE'].str.split(' ').str.get(0).astype(int)
    # Departements
    department = pd.read_excel(r"C:\Users\user\Downloads\oeh.medical.physician.xls")
    mydict = dict(zip(department['Name'].str.split(' ').str.get(-1), department['Speciality']))
    try:
        fh['Department'] = fh['DOCTOR'].str.split().str.get(-1)
    except:
        fh['Department'] = fh['SURGEON'].str.split().str.get(-1)
    fh['Department'] = fh.Department.replace(mydict)
    fh['Department'] = fh.Department.replace('Abboud', 'Obstetrics & Gynecology')
    # Date handling
    fh['DATE'] = pd.to_datetime(fh.DATE)
    fh['month'] = fh['DATE'].apply(lambda x : x.month)
    fh['year'] = fh['DATE'].apply(lambda x: x.year)
    fh['month'] = fh['month'].apply(lambda x : calendar.month_abbr[x])
    return fh

In [186]:
def counttext(column):
    column = pd.Series(column).dropna()
    column = list(chain(*column))
    counter = Counter(column)
    return counter

In [187]:
def dgdmydict( fh, analyzecolumn, filtercolumn, filteritem, DELIMETER):
    fh1 = fh[fh[filtercolumn] == filteritem]
    return counttext(fh1[analyzecolumn].str.lower().str.split(DELIMETER))

In [203]:
def analyzenow(fh, analyzecolumn, filtercolumn, filteritem):
    if analyzecolumn == 'GENDER':
        DELIMETER = " "
    elif analyzecolumn == 'COMPLAIN':
        DELIMETER = " + "
    elif analyzecolumn == 'IMAGING':
        DELIMETER = ","
    elif analyzecolumn == 'INVESTIGATION':
        DELIMETER = ","
    elif analyzecolumn == 'DOCTOR':
        DELIMETER = ""
    elif analyzecolumn == 'SURGEON':
        DELIMETER = ","
    else:
        DELIMETER = "9999999999999999"
    newfh = fh[analyzecolumn].replace(' ', np.nan)
    newfh = fh[analyzecolumn].dropna()
    want = dgdmydict(fh, analyzecolumn, filtercolumn, filteritem, DELIMETER)
    thedict = dict(want.most_common())
    df = pd.DataFrame(want.most_common())
    df = df.rename(columns={0 : analyzecolumn, 1 : "Counts"})
    return df

# Generate Presentation

In [204]:
def onegraph(fh, prs):
    _slide_layout = prs.slide_layouts[5]
        ###
    slide = prs.slides.add_slide(_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    title_shape.text = fh.columns[0].capitalize() + ' per {}'.format(fh.index.name.capitalize())
    print(fh.index.name)
    chart_data = CategoryChartData()
    chart_data.categories = [l for l in fh.index]
    chart_data.add_series('Series 1', (k for k in fh[fh.columns[0]]))
    # add chart to slide --------------------
    x, y, cx, cy = Inches(1), Inches(3), Inches(11), Inches(4)
    graphic_frame = slide.shapes.add_chart(
        XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data
    )
    chart = graphic_frame.chart
    #Categories
    category_axis = chart.category_axis
    category_axis.tick_labels.font.italic = True
    category_axis.tick_labels.font.size = Pt(15)
    

    chart.chart_title.has_text_frame=True
    chart.chart_title.text_frame.text= fh.columns[0] + ' ' + f'{fh.sum().Counts}'
    #Value axis
    value_axis = chart.value_axis
    #value_axis.maximum_scale = 50.0
    #Labels
    plot = chart.plots[0]
    plot.has_data_labels = True
    data_labels = plot.data_labels
    data_labels.font.size = Pt(17)
    data_labels.font.color.rgb = RGBColor(0x0A, 0x42, 0x80)
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END

In [205]:
# Loop through rows
def presentation(fh, prs):
    _slide_layout = prs.slide_layouts[5]
    for i in range(len(fh.index)):
        slide = prs.slides.add_slide(_slide_layout)
        shapes = slide.shapes
        title_shape = shapes.title
        title_shape.text = '{}'.format(fh.index[i])
        
        # define chart data ---------------------
        chart_data = CategoryChartData()
        chart_data.categories = [l for l in fh.columns[1:]]
        #chart_data.add_series('Series 1', (k for k in fh[i])) i for i in 
        chart_data.add_series('Series 1', (k for k in fh.iloc[i, 1:]))
        # add chart to slide --------------------
        x, y, cx, cy = Inches(1), Inches(3), Inches(11), Inches(4)
        graphic_frame = slide.shapes.add_chart(
            XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data
        )
        #print([l for l in fh.index[1:]])
        #print(k for k in fh[i][1:])
        chart = graphic_frame.chart
        #Categories
        category_axis = chart.category_axis
        category_axis.tick_labels.font.italic = True
        category_axis.tick_labels.font.size = Pt(15)

        chart.chart_title.has_text_frame=True
        chart.chart_title.text_frame.text= 'Total Patients {}'.format(fh['No/P'][i])
        #Value axis
        value_axis = chart.value_axis
        #value_axis.maximum_scale = 50.0
        
        #Labels
        plot = chart.plots[0]
        plot.has_data_labels = True
        data_labels = plot.data_labels
        data_labels.font.size = Pt(17)
        data_labels.font.color.rgb = RGBColor(0x0A, 0x42, 0x80)
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END

In [227]:
# OT
def ot():
    prs = Presentation(r"C:\Users\user\Documents\2020-Kalkaal Speciality Hospital\Monthly Presentations\OT\New Microsoft PowerPoint Presentation.pptx")
    PATH = r"C:\Users\user\Downloads\Surgery Patient Statistical Report (3).xlsx"
    OT_DATA = filehandle(PATH, "Report")
    cols = [ 'CASE', 'month', 'SURGEON' ]
    for c in cols:
        dataframe1 = pd.DataFrame()
        for i in OT_DATA[c].unique():
            dataframe = analyzenow(OT_DATA, c, c , i)
            dataframe['metadata'] = i
            dataframe1 = dataframe1.append(dataframe)
        lessthanone = pd.DataFrame(dataframe1[dataframe1['Counts'] <= 1])
        print(lessthanone.dropna().sum())
        grtthanone = dataframe1[dataframe1['Counts'] > 1]
        onegraph(dataframe1.set_index(c), prs)
    prs.save(r"C:\Users\user\Documents\2020-Kalkaal Speciality Hospital\Monthly Presentations\OT\Surgery Patient Statistical Presentation.pptx")

In [228]:
# Lab
def lab():
    folder = r"C:\Users\user\Documents\2020-Kalkaal Speciality Hospital\Abdullah\Daily Data"
    now = datetime.now()
    today = now.strftime("%d %B")
    today = "31 October Per Doctors"
    columns = ["DOCTOR", 'No/P', 'Biochemistry', 'Hormone', 'hematology', 'serology', 'Urine', 'Stool', 'Coag', 'Semen', 'Swab', 'culture']
    bigdata = pd.DataFrame(columns)
    #search file
    rawdata = r"C:\Users\user\Documents\2020-Kalkaal Speciality Hospital\Abdullah\Daily Data\31October Per Doctors.xlsx"
    try:
        fh = pd.read_excel(rawdata, sheet_name = 'Sheet1 (2)')
    except Exception as e:
        print(e, " in ", name)
    #data cleaning
    fh.drop_duplicates(inplace = True)
    fh = fh.drop([1])
    fh = fh.dropna()
    fh = fh.set_index("DOCTOR")
    print(fh)      
    fh.drop_duplicates(inplace = True)
    #fh = fh.T
    prs = Presentation(r"C:\Users\user\Documents\2020-Kalkaal Speciality Hospital\Abdullah\Presentation1.pptx")
    onegraph(fh, prs)
    presentation(fh, prs)
    prs.save("C:/Users/user/Documents/2020-Kalkaal Speciality Hospital/Abdullah/Presentation {}.pptx".format(today))
    

In [229]:
ot()

CASE        surgical managment of epistaxisamputationcorre...
Counts                                                     44
metadata    Surgical managment of epistaxisAmputationCorre...
dtype: object
CASE
month       0.0
Counts      0.0
metadata    0.0
dtype: float64
month
SURGEON     dr. haitham khadour
Counts                        1
metadata    Dr. Haitham Khadour
dtype: object
SURGEON
